# Youtube Data API v3 を使用して特定のチャンネルの動画とサムネイルを取得する

## Youtube API で動画に関する情報を取得
チャンネルを指定する際はChannel IDが必要になる。一般的には`https://www.youtube.com/channel/XXXXXXXXX`と表示されるが、カスタムURLを設定しているチャンネルは`https://www.youtube.com/c/XXXXXXXX`と表示される。  
カスタムURLを設定しているチャンネルを対象とする場合は、最初にチャンネルIDを調べる必要がある。

In [16]:
from googleapiclient.discovery import build
from oauth2client.tools import argparser

### Channel IDを調べる

In [4]:
DEVELOPER_KEY = <自分のGCP Developer key>
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
CUSTOM_NAME = "neet-tokyo" #チャンネルのcustom url 例：https://www.youtube.com/c/<ここ>

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
  developerKey=DEVELOPER_KEY)

search_response = youtube.search().list(
  q=CUSTOM_NAME,
  part="snippet",
  type='channel',
  maxResults=1,
  order="relevance", #関連順にソート
  pageToken=''
  ).execute()

print(search_response.get("items", [])[0])

{'kind': 'youtube#searchResult', 'etag': '6FcjWYFmqg_d9FvawDj6qA-Tuiw', 'id': {'kind': 'youtube#channel', 'channelId': 'UC4kUV8uGKmoQUs1HLzf7jsg'}, 'snippet': {'publishedAt': '2017-09-14T10:22:40Z', 'channelId': 'UC4kUV8uGKmoQUs1HLzf7jsg', 'title': 'ニートtokyo', 'description': 'ヒップホップコミュニティは必見！ ノーカットのショートインタビュー番組「ニートtokyo」 毎晩9時ぐらいにyoutubeで配信中！ 【Twitter】https://twitter.com/neettokyo ...', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AKedOLSFnAeY3zgubPxxca3DUGVjIGOzIdQTKXzjAzdz=s88-c-k-c0xffffffff-no-rj-mo'}, 'medium': {'url': 'https://yt3.ggpht.com/ytc/AKedOLSFnAeY3zgubPxxca3DUGVjIGOzIdQTKXzjAzdz=s240-c-k-c0xffffffff-no-rj-mo'}, 'high': {'url': 'https://yt3.ggpht.com/ytc/AKedOLSFnAeY3zgubPxxca3DUGVjIGOzIdQTKXzjAzdz=s800-c-k-c0xffffffff-no-rj-mo'}}, 'channelTitle': 'ニートtokyo', 'liveBroadcastContent': 'upcoming', 'publishTime': '2017-09-14T10:22:40Z'}}


### Channel IDから動画とサムネイルのURLを取得

In [5]:
DEVELOPER_KEY = <自分のGCP Developer key>
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
ChannelId = "UC4kUV8uGKmoQUs1HLzf7jsg" #上で出力したチャンネルID
videos = [] #動画のURLを格納
thums = [] #サムネイルのURLを格納

def youtube_search(pagetoken):
  youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  search_response = youtube.search().list(
    q=CUSTOM_NAME,
    part="snippet",
    channelId= ChannelId,
    maxResults=50,
    order="viewCount", #日付順にソート
    pageToken=pagetoken
    ).execute()


  for search_result in search_response.get("items", []):
    if search_result["id"]["kind"] == "youtube#video":
      videos.append(search_result["id"]["videoId"])
      thums.append(search_result["snippet"]["thumbnails"]["high"]["url"])

  try:
      nextPagetoken =  search_response["nextPageToken"] #nextPageTokenが返ってくる限り処理を繰り返す
      youtube_search(nextPagetoken)
  except:
      return

youtube_search('')

In [6]:
print(len(videos))
print(len(thums))

522
522


In [7]:
videos[:3]

['g0PDLoW9_ps', 'rvBp_qOM0L0', '4fiZA4FyhjM']

In [8]:
thums[:3]

['https://i.ytimg.com/vi/g0PDLoW9_ps/hqdefault.jpg',
 'https://i.ytimg.com/vi/rvBp_qOM0L0/hqdefault.jpg',
 'https://i.ytimg.com/vi/4fiZA4FyhjM/hqdefault.jpg']

# 動画から画像を切り出す

In [9]:
pip install youtube_dl

Note: you may need to restart the kernel to use updated packages.


In [14]:
from __future__ import unicode_literals
import youtube_dl

import cv2
import os 
import glob
from tqdm import tqdm

In [18]:
import wandb

In [20]:
wandb.init(project="neet tokyo")

OUTPUT_PATH = './dataset'
N_IMGS = 20


for video_id in tqdm(videos[14+89+106:250]):
  # 動画ダウンロード
  ydl_opts = {}
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([f'https://www.youtube.com/watch?v={video_id}'])

  video_paths1 = glob.glob('*.mp4')
  video_paths2 = glob.glob('*.mkv')
  video_paths = video_paths1 + video_paths2

  if len(video_paths) == 1:
    video_path = video_paths[0]
  else :
    for v in video_paths:
      os.remove(v)
    continue

  root, ext = os.path.splitext(video_path)
  basename = os.path.basename(root)
  output_dir = os.path.join(OUTPUT_PATH, basename)

  os.makedirs(output_dir, exist_ok=True)

  # 動画のカット
  cap = cv2.VideoCapture(video_path)

  cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
  fps = cap.get(cv2.CAP_PROP_FPS)

  video_len = frame_count / fps

  # 抽出したい開始or終了時間
  begin = 5
  end = video_len - 22

  # 切り出すフレーム
  cutout_interval = (end - begin) * fps / N_IMGS
  cutout_frames = [int(begin*fps + i*cutout_interval) for i in range(N_IMGS)]

  for frame_num in cutout_frames:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(os.path.join(output_dir, f'sample_{frame_num}.jpg'), frame)

  os.remove(video_path)


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



  0%|                                                                                                                                                                                 | 0/41 [00:00<?, ?it/s]

[youtube] xg3F9K4Ivew: Downloading webpage
[download] Destination: ¥ellow Bucks：ラップ以前の特技-xg3F9K4Ivew.f137.mp4
[download] 100% of 8.44MiB in 02:1842KiB/s ETA 00:008
[download] Destination: ¥ellow Bucks：ラップ以前の特技-xg3F9K4Ivew.f140.m4a
[download] 100% of 1016.88KiB in 00:1687KiB/s ETA 00:007
[ffmpeg] Merging formats into "¥ellow Bucks：ラップ以前の特技-xg3F9K4Ivew.mp4"
Deleting original file ¥ellow Bucks：ラップ以前の特技-xg3F9K4Ivew.f137.mp4 (pass -k to keep)
Deleting original file ¥ellow Bucks：ラップ以前の特技-xg3F9K4Ivew.f140.m4a (pass -k to keep)


  2%|████                                                                                                                                                                  | 1/41 [02:37<1:45:18, 157.96s/it]

[youtube] bYtpxNsyNz4: Downloading webpage
[download] Destination: DOGMA & JNKMN  - 人生でやめて一番よかったこと 〜'Authority'のフリースタイルに喰らった思い出(JNKMN)-bYtpxNsyNz4.f137.mp4
[download] 100% of 34.35MiB in 00:0169MiB/s ETA 00:00known ETA
[download] Destination: DOGMA & JNKMN  - 人生でやめて一番よかったこと 〜'Authority'のフリースタイルに喰らった思い出(JNKMN)-bYtpxNsyNz4.f140.m4a
[download] 100% of 1.82MiB in 00:0048MiB/s ETA 00:002
[ffmpeg] Merging formats into "DOGMA & JNKMN  - 人生でやめて一番よかったこと 〜'Authority'のフリースタイルに喰らった思い出(JNKMN)-bYtpxNsyNz4.mp4"
Deleting original file DOGMA & JNKMN  - 人生でやめて一番よかったこと 〜'Authority'のフリースタイルに喰らった思い出(JNKMN)-bYtpxNsyNz4.f137.mp4 (pass -k to keep)
Deleting original file DOGMA & JNKMN  - 人生でやめて一番よかったこと 〜'Authority'のフリースタイルに喰らった思い出(JNKMN)-bYtpxNsyNz4.f140.m4a (pass -k to keep)


...

[download] Destination: CHEHON  - 実演をお願いできますか？-4p1sP480FPk.f248.webm
[download] 100% of 7.64MiB in 00:0005MiB/s ETA 00:009
[download] Destination: CHEHON  - 実演をお願いできますか？-4p1sP480FPk.f140.m4a
[download] 100% of 1.21MiB in 00:0083MiB/s ETA 00:001
[ffmpeg] Merging formats into "CHEHON  - 実演をお願いできますか？-4p1sP480FPk.mkv"
Deleting original file CHEHON  - 実演をお願いできますか？-4p1sP480FPk.f248.webm (pass -k to keep)
Deleting original file CHEHON  - 実演をお願いできますか？-4p1sP480FPk.f140.m4a (pass -k to keep)


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 40/41 [48:01<01:30, 90.04s/it]

[youtube] lsz-bABV7dU: Downloading webpage


[download] Destination: BASE：朝ご飯昼ご飯晩ご飯どれが好き？-lsz-bABV7dU.f248.webm
[download] 100% of 15.49MiB in 03:1950KiB/s ETA 00:001
[download] Destination: BASE：朝ご飯昼ご飯晩ご飯どれが好き？-lsz-bABV7dU.f140.m4a
[download] 100% of 1.96MiB in 00:2558KiB/s ETA 00:009
[ffmpeg] Merging formats into "BASE：朝ご飯昼ご飯晩ご飯どれが好き？-lsz-bABV7dU.mkv"
Deleting original file BASE：朝ご飯昼ご飯晩ご飯どれが好き？-lsz-bABV7dU.f248.webm (pass -k to keep)
Deleting original file BASE：朝ご飯昼ご飯晩ご飯どれが好き？-lsz-bABV7dU.f140.m4a (pass -k to keep)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [51:52<00:00, 75.91s/it]
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/hypknot/neet%20tokyo/31nj8rtb/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/hypknot/neet%20tokyo/31nj8rtb/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/hypknot/neet%20tokyo/31nj8rtb/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: ht